In [2]:
# activate R magic
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


#**Classificação com Naive Bayes** - Filtrando mensagens de Spam via SMS
http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/

Problemas com a acentuação: https://support.rstudio.com/hc/en-us/articles/200532197-Character-Encoding



---

In [ ]:
%%R
# Pacotes
install.packages("slam")
install.packages("tm") # pacote de text mining
install.packages("SnowballC") 
install.packages("wordcloud") # construir uma nuvem de palavras
install.packages("gmodels") # constuir a confusions matrix
install.packages("e1071") # contém o algoritmo naive bayes

library(slam)
library(tm)
library(SnowballC)
library(wordcloud)
library(e1071)
library(gmodels)


##Carregando dados

In [5]:
%%R
dados <- read.csv("sms_spam.csv", stringsAsFactors = FALSE)

##Examinando estrutura de dados

In [6]:
%%R
str(dados)

'data.frame':	5559 obs. of  2 variables:
 $ type: chr  "ham" "ham" "ham" "spam" ...
 $ text: chr  "Hope you are having a good week. Just checking in" "K..give back my thanks." "Am also doing in cbe only. But have to pay." "complimentary 4 STAR Ibiza Holiday or £10,000 cash needs your URGENT collection. 09066364349 NOW from Landline "| __truncated__ ...


##Convertendo a variável para fator

In [7]:
%%R
dados$type <- factor(dados$type)
# Factor w/2 levels "ham", "spam" 1 1 1 2 2 1 1 1 2 1 ...

##Examinando a estrutura dos dados

In [9]:
%%R
str(dados$type)

 Factor w/ 2 levels "ham","spam": 1 1 1 2 2 1 1 1 2 1 ...


In [11]:
%%R
table(dados$type)


 ham spam 
4812  747 


##Construindo Corpus - conjunto de documentos de texto

In [16]:
%%R
# função VCorpus
dados_corpus <- VCorpus(VectorSource(dados$text))
# Large VCorpus (5559 elements, 20.4 Mb)

##Examinando a estrutura dos dados

In [14]:
%%R
print(dados_corpus)

<<VCorpus>>
Metadata:  corpus specific: 0, document level (indexed): 0
Content:  documents: 5559


In [17]:
%%R
inspect(dados_corpus[1:2])

<<VCorpus>>
Metadata:  corpus specific: 0, document level (indexed): 0
Content:  documents: 2

[[1]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 49

[[2]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 23



## Ajustando a estrutura

In [18]:
%%R
# convertendo p/ caracter
as.character(dados_corpus[[1]])

# aplica a função para cada um dos elementos
lapply(dados_corpus[1:2], as.character)

$`1`
[1] "Hope you are having a good week. Just checking in"

$`2`
[1] "K..give back my thanks."



## Limpeza do Corpus com tm_map()

In [19]:
%%R
#tm_map() - realiza transformações em corpus, convertendo todos os caracteres para tolower
dados_corpus_clean <- tm_map(dados_corpus, content_transformer(tolower))

## Diferenças entre Corpus inicial e Corpus após limpeza

Dando padrão aos elementos para auxiliar no processamento do algoritmo


In [20]:
%%R
as.character(dados_corpus[[1]])

[1] "Hope you are having a good week. Just checking in"


In [21]:
%%R
as.character(dados_corpus_clean[[1]])

[1] "hope you are having a good week. just checking in"


## Outras etapas de limpeza com tm_map

In [ ]:
%%R
dados_corpus_clean <- tm_map(dados_corpus_clean, removeNumbers) # remove números

In [ ]:
%%R
dados_corpus_clean <- tm_map(dados_corpus_clean, removeWords, stopwords()) # remove stop words, artigos, ligação de frase

In [ ]:
%%R
dados_corpus_clean <- tm_map(dados_corpus_clean, removePunctuation) # remove pontuação, focar no texto

---

##Criando uma função para **substituir** ao invés de remover pontuação

In [28]:
%%R
removePunctuation("hello...world")

[1] "helloworld"


In [29]:
%%R
# substituir ... por espaço com gsub
replacePunctuation <- function(x) { gsub("[[:punct:]]+", " ", x) }

In [30]:
%%R
replacePunctuation("hello...world")

[1] "hello world"


##Word stemming - remover gerúndio e manter padrão de palavras

In [31]:
%%R
#?wordStem
wordStem(c("learn", "learned", "learning", "learns"))

[1] "learn" "learn" "learn" "learn"


## Aplicando Stem

In [33]:
%%R
dados_corpus_clean <- tm_map(dados_corpus_clean, stemDocument)

##Eliminando espaço em branco desnecessário do conjunto

In [35]:
%%R
dados_corpus_clean <- tm_map(dados_corpus_clean, stripWhitespace) 

## Examinando a versão final do Corpus

In [36]:
%%R
lapply(dados_corpus[1:3], as.character)
lapply(dados_corpus_clean[1:3], as.character)

$`1`
[1] "hope good week just check"

$`2`
[1] "kgive back thank"

$`3`
[1] "also cbe pay"



## Criando uma matriz esparsa document-term - facilitar a construção de gráficos

In [39]:
%%R
#?DocumentTermMatrix - converter o conjunto numa matriz esparsa
dados_dtm <- DocumentTermMatrix(dados_corpus_clean)

## Solução alternartiva 2 - cria uma matriz esparsa document-term direto a partir do Corpus

Aumentou consideravelmente o número de termos. 

Indica que não conseguiu remover boa parte das stop-words, podendo ser considerado o pior procedimento.


In [40]:
%%R
# parâmetros p/ interferir na forma que a matriz será criada. 
dados_dtm2 <- DocumentTermMatrix(dados_corpus, control = list(tolower = TRUE, 
                                                              removeNumbers = TRUE, 
                                                              stopwords = TRUE, 
                                                              removePunctuation = TRUE,
                                                              stemming = TRUE))

## Solução alternativa 3 - usando stop words customizadas a partir da função

In [41]:
%%R
# função específica p/ remover stop words
dados_dtm3 <- DocumentTermMatrix(dados_corpus, control = list(tolower = TRUE,
                                                              removeNumbers = TRUE,
                                                              stopwords = function(x) { removeWords(x, stopwords()) },
                                                              removePunctuation = TRUE,
                                                              stemming = TRUE))

## Comparando resultados

In [43]:
%%R
dados_dtm


<<DocumentTermMatrix (documents: 5559, terms: 6537)>>
Non-/sparse entries: 42123/36297060
Sparsity           : 100%
Maximal term length: 40
Weighting          : term frequency (tf)


dados_dtm2 - pior procedimento aplicada, vide alto número de termos

In [44]:
%%R
dados_dtm2


<<DocumentTermMatrix (documents: 5559, terms: 6961)>>
Non-/sparse entries: 43221/38652978
Sparsity           : 100%
Maximal term length: 40
Weighting          : term frequency (tf)


In [45]:
%%R
dados_dtm3

<<DocumentTermMatrix (documents: 5559, terms: 6559)>>
Non-/sparse entries: 42147/36419334
Sparsity           : 100%
Maximal term length: 40
Weighting          : term frequency (tf)


## Criando subset de Treino


In [51]:
%%R
dados_dtm_train <- dados_dtm[1:4169, ]


## Criando subset de Teste

In [52]:
%%R
dados_dtm_test  <- dados_dtm[4170:5559, ]

##Labels (variável target)

In [54]:
%%R
dados_train_labels <- dados[1:4169, ]$type

In [55]:
%%R
dados_test_labels  <- dados[4170:5559, ]$type

##Verificando se a proporção de Spam é similar

In [56]:
%%R
prop.table(table(dados_train_labels))

dados_train_labels
      ham      spam 
0.8647158 0.1352842 


In [57]:
%%R
prop.table(table(dados_test_labels))

dados_test_labels
      ham      spam 
0.8683453 0.1316547 


## Word Cloud

In [ ]:
%%R
wordcloud(dados_corpus_clean, min.freq = 50, random.order = FALSE)

## Frequência dos dados

In [58]:
%%R
sms_dtm_freq_train <- removeSparseTerms(dados_dtm_train, 0.999)
sms_dtm_freq_train

<<DocumentTermMatrix (documents: 4169, terms: 1105)>>
Non-/sparse entries: 24836/4581909
Sparsity           : 99%
Maximal term length: 19
Weighting          : term frequency (tf)


##Indicador de Features para palavras frequentes

In [59]:
%%R
# Lista de palavras mais frequentes
findFreqTerms(dados_dtm_train, 5)

   [1] "£wk"                 "€˜m"                 "€˜s"                
   [4] "abiola"              "abl"                 "abt"                
   [7] "accept"              "access"              "accid"              
  [10] "account"             "across"              "act"                
  [13] "activ"               "actual"              "add"                
  [16] "address"             "admir"               "adult"              
  [19] "advanc"              "affect"              "aft"                
  [22] "afternoon"           "age"                 "ago"                
  [25] "aha"                 "ahead"               "aight"              
  [28] "aint"                "air"                 "aiyo"               
  [31] "alex"                "almost"              "alon"               
  [34] "alreadi"             "alright"             "also"               
  [37] "alway"               "angri"               "announc"            
  [40] "anoth"               "answer"              

## save frequently-appearing terms to a character vector


In [60]:
%%R
sms_freq_words <- findFreqTerms(dados_dtm_train, 5)
str(sms_freq_words)

 chr [1:1139] "£wk" "€˜m" "€˜s" "abiola" "abl" "abt" "accept" "access" ...


### Criando subsets apenas com palavras mais frequentes


In [61]:
%%R
sms_dtm_freq_train <- dados_dtm_train[ , sms_freq_words]
sms_dtm_freq_test <- dados_dtm_test[ , sms_freq_words]

## Converter para fator

In [62]:
%%R
convert_counts <- function(x) {
  print(x)
  x <- ifelse(x > 0, "Yes", "No")
}

## apply() converte counts para colunas de dados de treino e de teste


In [ ]:
%%R
sms_train <- apply(sms_dtm_freq_train, MARGIN = 2, convert_counts)
sms_test  <- apply(sms_dtm_freq_test, MARGIN = 2, convert_counts)

   1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16 
   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 
  17   18   19   20   21   22   23   24   25   26   27   28   29   30   31   32 
   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 
  33   34   35   36   37   38   39   40   41   42   43   44   45   46   47   48 
   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 
  49   50   51   52   53   54   55   56   57   58   59   60   61   62   63   64 
   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 
  65   66   67   68   69   70   71   72   73   74   75   76   77   78   79   80 
   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 
  81   82   83   84   85   86   87   88   89   90   91   92   93   94   95   96 
   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 
  97   98   99  100  101  10

In [ ]:
%%R
str(sms_train)

In [ ]:
%%R
str(sms_test)

##Treinando o modelo

In [ ]:
%%R
#?naiveBayes
nb_classifier <- naiveBayes(sms_train, dados_train_labels)

## Avaliando o modelo

In [ ]:
%%R
# Avaliando o modelo
sms_test_pred <- predict(nb_classifier, sms_test)

##Confusion Matrix

In [ ]:
%%R
CrossTable(sms_test_pred, 
           dados_test_labels,
           prop.chisq = FALSE, 
           prop.t = FALSE, 
           prop.r = FALSE,
           dnn = c('Previsto', 'Observado'))

## Melhorando a performance do modelo aplicando suavização laplace


In [ ]:
%%R
nb_classifier_v2 <- naiveBayes(sms_train, dados_train_labels, laplace = 1)

## Avaliando o modelo

In [ ]:
%%R
sms_test_pred2 <- predict(nb_classifier_v2, sms_test)

## Confusion Matrix


In [ ]:
%%R
CrossTable(sms_test_pred2, 
           dados_test_labels,
           prop.chisq = FALSE, 
           prop.t = FALSE, 
           prop.r = FALSE,
           dnn = c('Previsto', 'Observado'))

# Nota: Para fazer novas previsões com o modelo treinado, gera uma nova massa de dados e aplicar a função predict().
